In [1]:
import pandas as pd
import numpy as np

dayAhead = pd.read_pickle('./dayahead.pkl')
realTime = pd.read_pickle('./realtime.pkl')
capacity = pd.read_pickle('./Power_Plant_Capacities.pkl')
weather = pd.read_pickle('./weather.pkl')

In [2]:
import calendar
def MonthToString(month):
    return calendar.month_name[month]
Real_Time = realTime.to_frame()
Real_Time['Day of Month'] = Real_Time.index.get_level_values('date').day
Real_Time['Month'] = Real_Time.index.get_level_values('date').month
Real_Time['Year'] = Real_Time.index.get_level_values('date').year
Real_Time['Day of Week'] = Real_Time.index.get_level_values('date').weekday
DOW = {0:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday',4:'Friday',5:'Saturday',6:'Sunday'}
Real_Time['Day Of Week String'] = Real_Time['Day of Week'].map(DOW)
Real_Time['Month Name'] = Real_Time['Month'].map(MonthToString)




In [3]:
import datetime
weather.index = pd.to_datetime(weather.index)

In [4]:
Real_Time_Weather = Real_Time.reset_index().merge(weather,left_on='date',right_on='date').set_index(['prop','date']).loc['mean']

In [5]:
Real_Time_All = Real_Time_Weather.reset_index().merge(capacity,left_on=['Month','Year'],right_on=['Month','Year']).set_index('date')

In [6]:
Real_Time_All.to_pickle('realtime_combined.pkl')

In [7]:
y = Real_Time_All['total']

In [8]:
X = Real_Time_All.drop(['total','Day of Month','Month','Year','Day of Week'],axis=1)
X

,region,Day Of Week String,Month Name,Redding AWND,Redding PRCP,Redding TAVG,Redding TMAX,Redding TMIN,Sacremento AWND,Sacremento PRCP,...,Los Angeles TMAX,Los Angeles TMIN,Hydro Capacity,Solar Capacity,Gas Capacity,Wind Capacity,Geothermal Capacity,Nuclear Capacity,Coal Capacity,Biomass Capacity
date,,,,,,,,,,,,,,,,,,,,,
2017-01-01,NP15,Sunday,January,6.5,0.0,44.0,52.0,33.0,7.6,0.0,...,58.0,42.0,15565.39,11533.56,44368.67,5989.19,2758.75,2393.0,1898.3,1092.09
2017-01-01,SP15,Sunday,January,6.5,0.0,44.0,52.0,33.0,7.6,0.0,...,58.0,42.0,15565.39,11533.56,44368.67,5989.19,2758.75,2393.0,1898.3,1092.09
2017-01-01,ZP26,Sunday,January,6.5,0.0,44.0,52.0,33.0,7.6,0.0,...,58.0,42.0,15565.39,11533.56,44368.67,5989.19,2758.75,2393.0,1898.3,1092.09
2017-01-01,PGAE,Sunday,January,6.5,0.0,44.0,52.0,33.0,7.6,0.0,...,58.0,42.0,15565.39,11533.56,44368.67,5989.19,2758.75,2393.0,1898.3,1092.09
2017-01-01,SCE,Sunday,January,6.5,0.0,44.0,52.0,33.0,7.6,0.0,...,58.0,42.0,15565.39,11533.56,44368.67,5989.19,2758.75,2393.0,1898.3,1092.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-31,ZP26,Monday,December,12.3,0.0,47.0,55.0,41.0,16.3,0.0,...,65.0,46.0,15565.39,12464.25,44591.65,6301.60,2795.15,2393.0,1898.3,1099.86
2018-12-31,PGAE,Monday,December,12.3,0.0,47.0,55.0,41.0,16.3,0.0,...,65.0,46.0,15565.39,12464.25,44591.65,6301.60,2795.15,2393.0,1898.3,1099.86
2018-12-31,SCE,Monday,December,12.3,0.0,47.0,55.0,41.0,16.3,0.0,...,65.0,46.0,15565.39,12464.25,44591.65,6301.60,2795.15,2393.0,1898.3,1099.86


In [9]:
one_hot = ['region','Day Of Week String','Month Name']
scale = [x for x in list(X.columns) if x not in one_hot]

In [10]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.impute import KNNImputer
from sklearn.svm import SVR

In [11]:
scalePipe = Pipeline([('impute',KNNImputer(missing_values=np.nan)),('scale',StandardScaler())])

In [12]:
Column_Transformer = ColumnTransformer(transformers=[('Scale',scalePipe,scale),('One Hot',OneHotEncoder(),one_hot)])

In [13]:
Parameters = {'C':[0.1,0.5,1,5,10,50]}
Grid_Search = GridSearchCV(SVR(),Parameters)

In [14]:
Day_Ahead = dayAhead.to_frame()
Day_Ahead['Day of Month'] = Day_Ahead.index.get_level_values('date').day
Day_Ahead['Month'] = Day_Ahead.index.get_level_values('date').month
Day_Ahead['Year'] = Day_Ahead.index.get_level_values('date').year
Day_Ahead['Day of Week'] = Day_Ahead.index.get_level_values('date').weekday
DOW = {0:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday',4:'Friday',5:'Saturday',6:'Sunday'}
Day_Ahead['Day Of Week String'] = Day_Ahead['Day of Week'].map(DOW)
Day_Ahead['Month Name'] = Day_Ahead['Month'].map(MonthToString)

In [15]:
Day_Ahead_Weather = Day_Ahead.reset_index().merge(weather,left_on='date',right_on='date').set_index(['prop','date']).loc['mean']
Day_Ahead_All = Day_Ahead_Weather.reset_index().merge(capacity,left_on=['Month','Year'],right_on=['Month','Year']).set_index('date')

In [16]:
Day_Ahead_All.to_pickle('dayahead_combined.pkl')

In [17]:
y2 = Day_Ahead_All['total']
X2 = Day_Ahead_All.drop(['total','Day of Month','Month','Year','Day of Week'],axis=1)

# Test if random sampling works well

In [18]:
Dates = list(set(list(X.index)))
Dates.sort()
Dates[::10]
X3_test = X.loc[Dates[::10]]
X3_train = X.drop(Dates[::10],axis=0)
y3_test = y.loc[Dates[::10]]
y3_train = y.drop(Dates[::10],axis=0)

In [19]:
pipe3 = Pipeline([('column',Column_Transformer),('grid',Grid_Search)])

In [20]:
Dates = list(set(list(X2.index)))
Dates.sort()
Dates[::10]
X4_test = X2.loc[Dates[::10]]
X4_train = X2.drop(Dates[::10],axis=0)
y4_test = y2.loc[Dates[::10]]
y4_train = y2.drop(Dates[::10],axis=0)

In [21]:
pipe4 = Pipeline([('column',Column_Transformer),('grid',Grid_Search)])

# Subtract drift, periodicity


In [22]:
import math
import numpy as np
from sklearn import base
class FourierTransformer(base.BaseEstimator, base.TransformerMixin):
    
    def __init__(self):
        pass
    # We will need these in transform()
    
    def fit(self, X, y=None):
        # This transformer doesn't need to learn anything about the data,
        # so it can just return self without any further processing
        return self
    
    def transform(self, X):
        X['Julian_Day'] = X.index.to_julian_date()
        X['sin1'] = np.sin(2*math.pi*X['Julian_Day'].astype(int)/365.25)
        X['cos1'] = np.cos(2*math.pi*X['Julian_Day'].astype(int)/365.25)
        X['sin2'] = np.sin(2*math.pi*X['Julian_Day'].astype(int)/7)
        X['cos2'] = np.cos(2*math.pi*X['Julian_Day'].astype(int)/7)
        X['sin3'] = np.sin(2*math.pi*X['Julian_Day'].astype(int)/182.625)
        X['cos3'] = np.cos(2*math.pi*X['Julian_Day'].astype(int)/182.625)
        X['sin4'] = np.sin(2*math.pi*X['Julian_Day'].astype(int)/3.5)
        X['cos4'] = np.cos(2*math.pi*X['Julian_Day'].astype(int)/3.5)
        # Return an array with the same number of rows as X and one
        # column for each in self.col_names
        return X[['sin1','cos1','sin2','cos2','sin3','cos3','sin4','cos4']].astype(float)

In [23]:
class Composite(base.BaseEstimator, base.RegressorMixin):
  def __init__(self, linear,nonlinear):
    self.linear = linear
    self.nonlinear = nonlinear
    self.fourier = FourierTransformer()

  def fit(self, X, y):
    X2 = self.fourier.fit_transform(X)
    self.linear.fit(X2,y)
    res = np.asarray(y) - np.asarray(self.linear.predict(X2))
    self.nonlinear.fit(X,res)
    return self

  def predict(self, X):
    # make predictions 
    X2 = self.fourier.transform(X)
    pred = np.asarray(self.linear.predict(X2))+np.asarray(self.nonlinear.predict(X))
    return list(pred)

  def score(self, X, y):
    pred = self.predict(X)
    ave_y = np.mean(y)
    y = np.asarray(y)
    r2 = 1-np.sum((y-pred)**2)/np.sum((y-ave_y)**2)
    # custom score implementation
    return r2

In [24]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.impute import KNNImputer
from sklearn.svm import SVR
Dates = list(set(list(X.index)))
Dates.sort()
Dates[::10]
X3_test = X.loc[Dates[::10]]
X3_train = X.drop(Dates[::10],axis=0)
y3_test = y.loc[Dates[::10]]
y3_train = y.drop(Dates[::10],axis=0)
pipe3 = Pipeline([('column',Column_Transformer),('SVR',SVR(C=50))])
composite = Composite(Ridge(),pipe3)
composite.fit(X3_train,y3_train)

/opt/conda/envs/data3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/opt/conda/envs/data3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/envs/data3/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

Composite(linear=Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                       max_iter=None, normalize=False, random_state=None,
                       solver='auto', tol=0.001),
          nonlinear=Pipeline(memory=None,
                             steps=[('column',
                                     ColumnTransformer(n_jobs=None,
                                                       remainder='drop',
                                                       sparse_threshold=0.3,
                                                       transformer_weights=None,
                                                       transformers=[('Scale',
                                                                      Pipeline(memory=None,
                                                                               steps=[('impute',
                                                                                       KNNImputer(...
                                                   

In [25]:
composite.score(X3_test,y3_test)

0.32801700166186054

In [26]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.impute import KNNImputer
from sklearn.svm import SVR
Dates = list(set(list(X2.index)))
Dates.sort()
Dates[::10]
X4_test = X2.loc[Dates[::10]]
X4_train = X2.drop(Dates[::10],axis=0)
y4_test = y2.loc[Dates[::10]]
y4_train = y2.drop(Dates[::10],axis=0)
pipe4 = Pipeline([('column',Column_Transformer),('SVR',SVR(C=50))])
composite2 = Composite(Ridge(),pipe4)
composite2.fit(X4_train,y4_train)

/opt/conda/envs/data3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/opt/conda/envs/data3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/envs/data3/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

Composite(linear=Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                       max_iter=None, normalize=False, random_state=None,
                       solver='auto', tol=0.001),
          nonlinear=Pipeline(memory=None,
                             steps=[('column',
                                     ColumnTransformer(n_jobs=None,
                                                       remainder='drop',
                                                       sparse_threshold=0.3,
                                                       transformer_weights=None,
                                                       transformers=[('Scale',
                                                                      Pipeline(memory=None,
                                                                               steps=[('impute',
                                                                                       KNNImputer(...
                                                   

In [27]:
composite2.score(X4_test,y4_test)

0.5843980967711122

In [28]:
import pickle
from joblib import dump, load

dump(composite,'realtime_model_Fourier.pkl')
dump(composite2,'dayahead_model_Fourier.pkl')

['dayahead_model_Fourier.pkl']